In [3]:
import pandas as pd
import os
import glob
# _ = os.stat(f'{os.path.expanduser("~")}/experiments/').st_mtime    # to refresh the cache

metric_files = glob.glob(f'{os.path.expanduser("~")}/experiments/*/logfile*')
minus_metric_files = []
minus_metric_files += glob.glob(f'{os.path.expanduser("~")}/experiments/*5000/logfile*')
minus_metric_files += glob.glob(f'{os.path.expanduser("~")}/experiments/dev/logfile*')
minus_metric_files += glob.glob(f'{os.path.expanduser("~")}/experiments/nsfp/logfile*')

metric_files = [x for x in metric_files if x not in minus_metric_files]

all_metrics = []

for i in range(len(metric_files)):
    file = metric_files[i]
    opened_file = open(file, 'r')

    accum_info = {}

    for line in opened_file.readlines():

        if 'dataset' in line:
            dataset = line.split('=')[-1].strip()
            dataset = dataset.capitalize()

        if 'exp_name' in line:
            exp_name = line.split('=')[-1].strip()

        if 'EPE3D' in line:
            epe3d = float(line.split(' ')[-1])

        if 'acc3d_strict' in line:
            acc3d_strict = float(line.split(' ')[-1])

        if 'acc3d_relax' in line:
            acc3d_relax = float(line.split(' ')[-1])

        if 'angle_error' in line:
            angle_error = float(line.split(' ')[-1])

        if 'outlier' in line:
            outlier = float(line.split(' ')[-1])

        if 'solver_time' in line:
            solver_time = float(line.split(' ')[-1])

    accum_info = {"exp_name" : exp_name,

                  "epe3d" : epe3d,
                  "dataset" : dataset,
                  "acc3d_strict" : acc3d_strict,
                  "acc3d_relax" : acc3d_relax,
                  "angle_error" : angle_error,
                  "outlier" : outlier,
                  "solver_time" : solver_time}

    all_metrics.append(accum_info)

    opened_file.close()

metric_df = pd.DataFrame(all_metrics)


### Works, highlights  rows
# highest_epe = metric_df['epe3d'].argmax()
# print(highest_epe)
# subsets = pd.IndexSlice[highest_epe, :]

# metric_df.style.applymap(lambda x: "background-color: green", subset=subsets)
metric_df


,exp_name,epe3d,dataset,acc3d_strict,acc3d_relax,angle_error,outlier,solver_time
0,forward_flow,0.092443,Argoverse,0.697132,0.874634,0.210007,0.125563,10.344284
1,visibility_smoothness,0.082483,Argoverse,0.565055,0.805950,0.101770,0.195077,16.699733
2,smoothness,0.097475,Argoverse,0.593866,0.820662,0.208463,0.179338,16.613856
3,forward_and_visibility,0.077181,Argoverse,0.614720,0.861715,0.096689,0.138285,9.628811
4,allec,0.093097,Argoverse,0.382482,0.655555,0.057583,0.345538,15.728599
5,baseline,0.116202,Argoverse,0.586211,0.797746,0.357751,0.461535,16.009291


In [5]:

from data.PATHS import TMP_VIS_PATH
import matplotlib.pyplot as plt
plt.close()
plt.rcParams["figure.figsize"] = (18,10)
plt.title("Model performance")
plt.grid(True)
plt.xlabel("Time Complexity (s)")
plt.ylabel("EPE3D [m]")

# annotate points
for i in range(len(metric_df)):
    exp_name = metric_df['exp_name'][i]
    if 'ours' in exp_name or 'visibility' in exp_name or 'forward' in exp_name:
        color = 'r'
    else:
        color = 'b'

    # plot marker
    plt.plot(metric_df['solver_time'][i], metric_df['epe3d'][i], f'{color}*', markersize=20)

                    # Name of the point                         # Coordinates
    plt.annotate(exp_name.replace('-ours','(ours)'), (metric_df['solver_time'][i], metric_df['epe3d'][i]), fontsize=15)

plt.savefig(TMP_VIS_PATH + '/gif.png')


In [6]:


# if k=5, as always
# if k=[5,6,7...], then trigger ablation? do that in a solver and store multiple files?

